<a href="https://colab.research.google.com/github/aaalexlit/omdena_climate_change_challenge_notebooks/blob/main/Full_pipeline_claim_verification_POC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Download the database of scientific warrants

Note:  
I've created this database on my local machine and then uploaded the resulting files to Google Drive

Breif description of the procedure (more detailed one to come):
1. Download abstracts and metadata from https://openalex.org/ using the following search terms 
```
"climate"|elevated%20temperature"|"ocean%20warming"|"saline%20intrusion"|"environmental%20change"|"global%20warming"|"global%20change"|"greenhouse%20effect"|"snow%20cover"|"extreme%20temperature"|"cyclone"|"ocean%20acidification"|"anthropogenic"|"sea%20level"|"precipitation%20variability"|"precipitation%20change"|"temperature%20impact"|"environmental%20variability"|"weather%20pattern"|"weather%20factor"|"increase%20in%20temperature"),title.search:"species"|"mortality"|"health"|"disease"|"ecosystem"|"mass%20balance"|"flood"|"drought"|"disease"|"adaptation"|"malaria"|"fire"|"water%20scarcity"|"water%20supply"|"permafrost"|"biological%20response"|"food%20availability"|"food%20security"|"vegetation%20dynamic"|"cyclone"|"yield"|"gender"|"indigenous"|"conflict"|"inequality"|"snow%20water%20equivalent"|"surface%20temperature"|"glacier%20melt"|"glacier%20mass"|"coastal%20erosion"|"glacier%20retreat"|"rainfall%20reduction"|"reduction%20in%20rainfall"|"coral%20stress"|"precipitation%20increase"|"precipitation%20decrease"|"river%20flow",title.search:"recent"|"current"|"modern"|"observation"|"observed"|"observable"%20"evidence"|"past"|"local"|"regional"|"significant"|"driver"|"driving"|"response"|"were%20responsible"|"was%20responsible"|"exhibited"|"witnessed"|"attribution"|"attributed"|"attributable"|"has%20increased"|"has%20decreased"|"historic"|"correlation"|"evaluation"
```
and searching against article titles
mainly using [this code](https://github.com/mcallaghan/NLP-climate-science-tutorial-CCAI/blob/main/A_obtaining_data.ipynb) with some modifications. The result is around 8000 abstracts
1. For every abstract split it into sentences with `nltk.tokenize.sent_tokenize` 
1. For every sentence decide if it's a check-worthy factual statement using [this model](https://huggingface.co/whispAI/ClaimBuster-DeBERTaV2)
1. If it is then index into the database along with the meta information and semantic embedding (**that also includes the title of the article along with the given phrase**) using the [general small-ish model from sentence transformers](
https://huggingface.co/sentence-transformers/all-MiniLM-L6-v2). *Spoiler alert:* I actually find the results quite reasonable


This procedure took around 8 hours of running (and about the same time of experimenting before I was actually able to run it)
So, certainly the code needs a performance improvement)

In [1]:
## we need this to be able to download big-ish files from GDrive
%%capture
!pip3 install --upgrade gdown

In [2]:
%%capture
!gdown https://drive.google.com/uc?id=1-6yyxt5o6RnTOONkzcssUxKqHcdK88cm -O faiss_document_store.db
!gdown https://drive.google.com/uc?id=1-4_yOR9341_6m7D2-9yOd-zINs9TAsZW -O faiss_index
!gdown https://drive.google.com/uc?id=13El8NchthvhIXBMxlGSTJLwjvwNPd5h7 -O faiss_index.json

## Uncomment the following to use the newer scientific evidences database with claim detection off (ie abstracts just split into phrases and indexed as is)

There are around 62000 sentences extracted from 10300 abstracts
found in openalex.org using keyword search on paper title
Created in the following notebook https://colab.research.google.com/drive/10VVw_LusJ5n0F1outygCl4ECyCK-M8zv?usp=sharing

In [3]:
!gdown https://drive.google.com/uc?id=1R47DFGuOmZlZYo2MBvCK8n3pZfHEMeRm -O faiss_document_store.db
!gdown https://drive.google.com/uc?id=1-GxzZOUqAWbY8de3iF_Qash4jWHvKwwD -O faiss_index
!gdown https://drive.google.com/uc?id=1-NecBRyDHwsinkwbw1DX5uDtcrcK6KdE -O faiss_index.json

Downloading...
From: https://drive.google.com/uc?id=1R47DFGuOmZlZYo2MBvCK8n3pZfHEMeRm
To: /content/faiss_document_store.db
100% 152M/152M [00:00<00:00, 227MB/s]
Downloading...
From: https://drive.google.com/uc?id=1-GxzZOUqAWbY8de3iF_Qash4jWHvKwwD
To: /content/faiss_index
100% 95.8M/95.8M [00:00<00:00, 162MB/s]
Downloading...
From: https://drive.google.com/uc?id=1-NecBRyDHwsinkwbw1DX5uDtcrcK6KdE
To: /content/faiss_index.json
100% 160/160 [00:00<00:00, 287kB/s]


## Install required libraries

### Check if GPU is available to install gpu version of faiss

In [4]:
import torch
import os
faiss_to_install = "faiss-gpu"
if not torch.cuda.is_available():
  faiss_to_install = "faiss"

ret_code = os.system(f"pip install farm-haystack[{faiss_to_install}]")
if not ret_code:
  print(f"Installed {faiss_to_install}")

Installed faiss-gpu


In [5]:
%%capture
!pip install sentence-transformers
!pip install jsonlines
!pip install newspaper3k

In [6]:
%%capture
!python -m nltk.downloader punkt

## Imports

In [7]:
from collections import defaultdict
import jsonlines
import pandas as pd
from transformers import AutoTokenizer, AutoModelForSequenceClassification, RobertaForSequenceClassification
from nltk.tokenize import sent_tokenize
import torch

from haystack.nodes import EmbeddingRetriever
from haystack.document_stores.faiss import FAISSDocumentStore
import logging

import newspaper
from newspaper import Article

from transformers import pipeline


## Haystack config
- logging config  
- model to use to create embeddings of the input phrases for semantic search against the DB

In [8]:
logging.basicConfig(format="%(levelname)s - %(name)s -  %(message)s", level=logging.CRITICAL)
SEMANTIC_EMBEDDING_MODEL_NAME = 'sentence-transformers/all-MiniLM-L6-v2'

## Download models for claim detection and verification and to check if a text is about climate

In [9]:
claimbuster_tokenizer = AutoTokenizer.from_pretrained("lucafrost/ClaimBuster-DeBERTaV2")
claimbuster_model = AutoModelForSequenceClassification.from_pretrained("lucafrost/ClaimBuster-DeBERTaV2")

climate_factcheck_tokenizer = AutoTokenizer.from_pretrained("amandakonet/climatebert-fact-checking")
climate_factcheck_model = AutoModelForSequenceClassification.from_pretrained("amandakonet/climatebert-fact-checking")

climatebert_tokenizer = AutoTokenizer.from_pretrained("climatebert/distilroberta-base-climate-f")
climate_attention_model = RobertaForSequenceClassification.from_pretrained('kruthof/climateattention-ctw',
                                                                           num_labels=2)

## Claim detection
Given an array of sentences and optionally a threshold decide if a sentence is a claim or not

In [10]:
def is_claim(sentences, model=claimbuster_model, tokenizer=claimbuster_tokenizer, debug=False):
    inputs = tokenizer(sentences,
                       padding=True,
                       truncation=True,
                       max_length=512,
                       return_tensors="pt")
    device = 0 if torch.cuda.is_available() else -1
    pipe = pipeline("text-classification", model=model,
                    tokenizer=tokenizer, device=device)
    labels, probs = [], []
    for out in pipe(sentences, batch_size=1):
        labels.append(out['label'])
        probs.append(out['score'])
    if debug:
        for sentence, label, prob in zip(sentences, labels, probs):
            print(f"{label}({prob:.3f})")
            print(sentence)
    return list(map(lambda l: model.config.label2id[l], labels)), probs


def tokenize_texts_to_sentences(text):
    return sent_tokenize(text)

In [11]:
def get_claims_from_text(text, threshold=0.5, debug=False):
    sentences = tokenize_texts_to_sentences(text)
    predicted_class_ids, probs = is_claim(sentences, debug=debug)
    return [sentence for sentence, label, prob in zip(sentences, predicted_class_ids, probs) if
            label in [1, 2] and prob > threshold]

## Check if a text is climate-related

In [12]:
def is_about_climate(texts):
    if torch.cuda.is_available():
      device = 0
      batch_size = 64
    else:
      device = -1
      batch_size = 1
    pipe = pipeline("text-classification", model=climate_attention_model,
                    tokenizer=climatebert_tokenizer, device=device,
                    truncation=True, padding=True)
    labels = []
    probs = []
    for out in pipe(texts, batch_size=batch_size):
        labels.append(out['label'])
        probs.append(out['score'])
    return labels, probs

In [13]:
def filter_climate_related(sentences, debug=False):
    labels, probs = is_about_climate(sentences)
    if debug:
      for sentence, label, prob in zip(sentences, labels, probs):
        print(f'{sentence}\nclimate related? {label}\nprob: {prob:.3f}')
    return [sentence for label, sentence in zip(labels, sentences) if label == 'Yes']

## Initialize core components of Semantic search

In [14]:
def get_faiss_document_store():
    return FAISSDocumentStore.load(index_path='/content/faiss_index')


def get_retriever(document_store):
    return EmbeddingRetriever(
        document_store=document_store,
        embedding_model=SEMANTIC_EMBEDDING_MODEL_NAME,
        model_format='sentence_transformers',
        # include article title into the embedding
        embed_meta_fields=["title"],
        progress_bar=False
    )

In [15]:
def retrieve_matches_for_a_phrase(phrase, top_k=10):
    doc_store = get_faiss_document_store()
    retriever = get_retriever(doc_store)
    return retriever.retrieve(phrase, top_k=top_k)

## Scrape the article from URL


In [16]:
def get_text_from_url(news_article_url):
  article = Article(news_article_url)
  article.download()
  article.parse()
  article.nlp()
  return article.text

## Retrieve potential evidence from the database for a given text 

In [17]:
def get_evidences_from_text(text, filter_climate=True, debug=False):
  if text.startswith('http'):
    text = get_text_from_url(text)
  if not filter_climate_related([text]):
    print("The text is not climate related")
    return []
  claims_from_text = get_claims_from_text(text)
  if filter_climate:
    claims_from_text = filter_climate_related(claims_from_text)
  res = []
  for claim in claims_from_text:
    evidences = retrieve_matches_for_a_phrase(claim)
    res.append((claim, evidences))
    if debug:
        print('Claim\n', claim)
        for i, evidence_doc in enumerate(evidences):
          print(f'Evidence {i}:\n',
                  f'Similarity: {evidence_doc.score:.3f}\n'
                  f'Quote: {evidence_doc.content}\n',
                  f'Article Title: {evidence_doc.meta.get("title", "")}\n',
                  f'DOI: {evidence_doc.meta.get("doi", "")}')
  return res

In [18]:
def convert_claim_evidence_to_text(claim_evidence_array):
  res = []
  for claim, evidences in claim_evidence_array:
    evidences_text = [evidence.content for evidence in evidences]
    res.append([claim, evidences_text])
  return res

## Do identified evidences support of refute the claims extracted from the article?

In [19]:
climate_factcheck_model.config.id2label = {
    0: "SUPPORTS",
    1: "REFUTES",
    2: "NOT_ENOUGH_INFO"
  }

def predict_supports_or_refutes(claim_evidence_array):
  def claim_evidence_pair_data():
    for claim, evidences in claim_evidence_array: 
      for evidence in evidences:
        yield {"text": claim, "text_pair": evidence}

  device = 0 if torch.cuda.is_available() else -1
  pipe = pipeline("text-classification", model=climate_factcheck_model,
                  tokenizer=climate_factcheck_tokenizer, device=device,
                  truncation=True, padding=True)
  labels = []
  probs = []
  for out in pipe(claim_evidence_pair_data(), batch_size=64):
      labels.append(out['label'])
      probs.append(out['score'])
  return labels, probs
  

In [20]:
def is_supported_by_science(text, print_neutral=False, threshold=0.5):
  claim_evidence_array = get_evidences_from_text(text)
  claim_evidence_array = convert_claim_evidence_to_text(claim_evidence_array)
  claim_evidence_pairs = list(zip(*[(claim, evidence) for claim, evidences in claim_evidence_array for evidence in evidences]))
  labels, probs = predict_supports_or_refutes(claim_evidence_array)
  for claim, evidence, label, prob in zip(*claim_evidence_pairs, labels, probs):
    if prob > threshold:
      if (label == 'NOT_ENOUGH_INFO' and print_neutral) or label != 'NOT_ENOUGH_INFO': 
        print(f'Label:\n{label}\nClaim:\n{claim}\nEvidence:\n{evidence}\nProb:\n{prob:.2f}\n')

##Try it out with some simple text

Using claims reviewed [here](https://climatefeedback.org/claimreview/the-epoch-times-prints-range-of-inaccurate-misleading-claims-climate-changes-impacts-causes-patrick-moore/)




In [21]:
is_supported_by_science(
    """CO2 is not the cause of our current warming trend.
    Arctic sea ice has expanded in recent years.
    Polar bears’ population is growing and is not threatened by climate change.
    CO2 is good for plant life""");

Label:
REFUTES
Claim:
CO2 is not the cause of our current warming trend.
Evidence:
Coal-fired power plants emit large amounts of CO 2 , which constitutes one the largest causes global warming.
Prob:
0.59

Label:
REFUTES
Claim:
CO2 is not the cause of our current warming trend.
Evidence:
recent decades, United Nations’ (WMO, UNEP) change actions, IPCC reports, other scientific publications strongly emphasized that CO2 increase was “very likely” triggering global warming, which already led negative consequences for environment society.
Prob:
0.66

Label:
REFUTES
Claim:
CO2 is not the cause of our current warming trend.
Evidence:
The radiative effect from given atmospheric CO2 perturbation same for fossil fuel combustion.
Prob:
0.51

Label:
REFUTES
Claim:
CO2 is not the cause of our current warming trend.
Evidence:
Nearly all climate scientists agree that main cause this rise is ever-increasing accumulations ‘greenhouse gases’, especially carbon dioxide and methane, within our atmosphere.

## Try out some bigger text
[Article Text Source](https://archive.is/mLGZE#selection-4563.0-4583.237)  
[Article Review on climatefeedback](https://climatefeedback.org/evaluation/wall-street-journal-steven-koonin-publishes-misleading-claims-climate-change-influences-greenland-ice-melt/)

In [22]:
is_supported_by_science("""Greenland’s Melting Ice Is No Cause for Climate-Change Panic.
The annual loss has been decreasing in the past decade even as the globe continues to warm.
One of the most sacred tenets of climate alarmism is that Greenland’s vast ice sheet is shrinking ever more rapidly because of human-induced climate change. The media and politicians warn constantly of rising sea levels that would swamp coastlines from Florida to Bangladesh. A typical headline: “Greenland ice sheet on course to lose ice at fastest rate in 12,000 years.”
With an area of 660,000 square miles and a thickness up to 1.9 miles, Greenland’s ice sheet certainly deserves attention. Its shrinking has been a major cause of recent sea-level rise, but as is often the case in climate science, the data tell quite a different story from the media coverage and the political laments.
The chart nearby paints a bigger picture that is well known to experts but largely absent from the media and even from the most recent United Nations climate report. It shows the amount of ice that Greenland has lost every year since 1900, averaged over 10-year intervals; the annual loss averages about 110 gigatons. (A gigaton is one billion metric tons, or slightly over 2.2 trillion pounds.) That is a lot, but that water has caused the planet’s oceans to rise each year by only 0.01 inch, about one-fifth the thickness of a dime.
In contrast, the United Nations’ Intergovernmental Panel on Climate Change projects that for the most likely course of greenhouse-gas emissions in the 21st century, the average annual ice loss would be somewhat larger than the peak values shown in the graph. That would cause sea level to rise by 3 inches by the end of this century, and if losses were to continue at that rate, it would take about 10,000 years for all the ice to disappear, causing sea level to rise more than 20 feet.
To assess the importance of human influences, we can look at how the rate of ice loss has changed over time.
In that regard, the graph belies the simplistic notion that humans are melting Greenland. Since human warming influences on the climate have grown steadily—they are now 10 times what they were in 1900— you might expect Greenland to lose more ice each year. Instead there are large swings in the annual ice loss and it is no larger today than it was in the 1930s, when human influences were much smaller. Moreover, the annual loss of ice has been decreasing in the past decade even as the globe continues to warm.
While a warming globe might eventually be the dominant cause of Greenland’s shrinking ice, natural cycles in temperatures and currents in the North Atlantic that extend for decades have been a much more important influence since 1900. Those cycles, together with the recent slowdown, make it plausible that the next few decades will see a further, perhaps dramatic slowing of ice loss. That would be inconsistent with the IPCC’s projection and wouldn’t at all support the media’s exaggerations.
Much climate reporting today highlights short-term changes when they fit the narrative of a broken climate but then ignores or plays down changes when they don’t, often dismissing them as “just weather.”
Climate unfolds over decades. Although short-term changes might be deemed news, they need to be considered in a many-decade context. Media coverage omitting that context misleadingly raises alarm. Greenland’s shrinking ice is a prime example of that practice.
If Greenland’s ice loss continues to slow, headline writers will have to find some other aspect of Greenland’s changes to grab our attention, and politicians will surely find some other reason to justify their favorite climate policies.""");


Label:
SUPPORTS
Claim:
The annual loss has been decreasing in the past decade even as the globe continues to warm.
Evidence:
Around the world, small ice caps and glaciers have been losing mass retreating since start of industrial era.
Prob:
0.58

Label:
SUPPORTS
Claim:
One of the most sacred tenets of climate alarmism is that Greenland’s vast ice sheet is shrinking ever more rapidly because of human-induced climate change.
Evidence:
New research finds that the Greenland environment is highly sensitive to recent warming trends.
Prob:
0.61

Label:
SUPPORTS
Claim:
One of the most sacred tenets of climate alarmism is that Greenland’s vast ice sheet is shrinking ever more rapidly because of human-induced climate change.
Evidence:
The dynamic loss of ice via outlet glaciers around the Greenland Ice Sheet is a major contributor to sea level rise.
Prob:
0.56

Label:
SUPPORTS
Claim:
One of the most sacred tenets of climate alarmism is that Greenland’s vast ice sheet is shrinking ever more rapid

## And Scraping directly from URL
eg
https://www.thesun.co.uk/news/15288148/climate-change-risks-economy/

In [23]:
is_supported_by_science("https://www.thesun.co.uk/news/15288148/climate-change-risks-economy/")

Label:
SUPPORTS
Claim:
CLIMATE change risks are growing but long, hot summers will boost our economy, experts believe.
Evidence:
Climate change is likely to have relevant effects on our future socio-economic systems.
Prob:
0.59

Label:
SUPPORTS
Claim:
CLIMATE change risks are growing but long, hot summers will boost our economy, experts believe.
Evidence:
Climate change is one of the biggest threat to human health and agriculture in recent years.
Prob:
0.64

Label:
SUPPORTS
Claim:
2 Climate change risks are growing but long, hot summers will boost economy, experts claim Credit: Alamy Live News

But it could also mean longer summers which will fuel a tourism boom.
Evidence:
Sea level rise, extreme events, loss of biodiversity, water stress among other direct and indirect impacts pose potential danger for the tourism industry.
Prob:
0.57

Label:
SUPPORTS
Claim:
But longer, hotter summers will lead to a tripling of heat-related deaths to around 7,000 per year.
Evidence:
Summer days with e

In [24]:
is_supported_by_science("https://www.nationalgeographic.com/environment/article/amazon-rainforest-now-appears-to-be-contributing-to-climate-change")

Label:
SUPPORTS
Claim:
For years, researchers have expressed concern that rising temperatures, drought, and deforestation are reducing the capacity of the world’s largest rainforest to absorb carbon dioxide from the atmosphere, and help offset emissions from fossil-fuel burning.
Evidence:
Abstract Background At present, global warming is an indisputable fact, and more attention has been paid to the impacts of climate on ecological environments.
Prob:
0.86

Label:
SUPPORTS
Claim:
For years, researchers have expressed concern that rising temperatures, drought, and deforestation are reducing the capacity of the world’s largest rainforest to absorb carbon dioxide from the atmosphere, and help offset emissions from fossil-fuel burning.
Evidence:
Forest inventories from the intact rainforests of Amazon indicate increasing rates carbon gain over past three decades.
Prob:
0.54

Label:
SUPPORTS
Claim:
For years, researchers have expressed concern that rising temperatures, drought, and deforesta

In [25]:
is_supported_by_science("https://news.sky.com/story/matt-hancock-responds-to-leak-of-lockdown-whatsapp-messages-12823380")

The text is not climate related


# Examples



## Claim detection (Somewhat okay)





In [26]:
get_claims_from_text("""CO2 is not the cause of our current warming trend.
    Arctic sea ice has expanded in recent years.
    Polar bears’ population is growing and is not threatened by climate change.
    CO2 is good for plant life""", threshold=0, debug=True)

Check-worthy Factual Statement (CFS)(0.647)
CO2 is not the cause of our current warming trend.
Check-worthy Factual Statement (CFS)(0.984)
Arctic sea ice has expanded in recent years.
Check-worthy Factual Statement (CFS)(0.984)
Polar bears’ population is growing and is not threatened by climate change.
Non-Factual Statement (NFS)(0.504)
CO2 is good for plant life


['CO2 is not the cause of our current warming trend.',
 'Arctic sea ice has expanded in recent years.',
 'Polar bears’ population is growing and is not threatened by climate change.']

From the following article abstract https://www.science.org/doi/10.1126/science.1239402

In [27]:
get_claims_from_text("""Climate change could potentially interrupt progress toward a world without hunger. 
A robust and coherent global pattern is discernible of the impacts of climate change on crop productivity that could have consequences for food availability. 
The stability of whole food systems may be at risk under climate change because of short-term variability in supply. 
However, the potential impact is less clear at regional scales, but it is likely that climate variability and change will exacerbate food insecurity in areas currently vulnerable to hunger and undernutrition. 
Likewise, it can be anticipated that food access and utilization will be affected indirectly via collateral effects on household and individual incomes, and food utilization could be impaired by loss of access to drinking water and damage to health. The evidence supports the need for considerable investment in adaptation and mitigation actions toward a “climate-smart food system” that is more resilient to climate change influences on food security.""", 
threshold=0, debug=True);

Non-Factual Statement (NFS)(0.970)
Climate change could potentially interrupt progress toward a world without hunger.
Non-Factual Statement (NFS)(0.904)
A robust and coherent global pattern is discernible of the impacts of climate change on crop productivity that could have consequences for food availability.
Non-Factual Statement (NFS)(0.906)
The stability of whole food systems may be at risk under climate change because of short-term variability in supply.
Non-Factual Statement (NFS)(0.872)
However, the potential impact is less clear at regional scales, but it is likely that climate variability and change will exacerbate food insecurity in areas currently vulnerable to hunger and undernutrition.
Non-Factual Statement (NFS)(0.945)
Likewise, it can be anticipated that food access and utilization will be affected indirectly via collateral effects on household and individual incomes, and food utilization could be impaired by loss of access to drinking water and damage to health.
Non-Fact

In [28]:
get_claims_from_text(get_text_from_url("https://www.thesun.co.uk/news/15288148/climate-change-risks-economy/"), threshold=0.5, debug=True);

Check-worthy Factual Statement (CFS)(0.551)
CLIMATE change risks are growing but long, hot summers will boost our economy, experts believe.
Non-Factual Statement (NFS)(0.475)
Increasingly warm weather threatens drought, flooding and other deadly disruption and urgent action is needed to deal with it, they say.
Check-worthy Factual Statement (CFS)(0.521)
2 Climate change risks are growing but long, hot summers will boost economy, experts claim Credit: Alamy Live News

But it could also mean longer summers which will fuel a tourism boom.
Check-worthy Factual Statement (CFS)(0.958)
Some experts predict this could boost GDP by up to 0.3 per cent a year.
Check-worthy Factual Statement (CFS)(0.621)
Bumper fruit crops and the ability to produce more British wine may be up­sides, the Climate Change Committee reports.
Non-Factual Statement (NFS)(0.874)
And spending more time outside will make us healthier, it suggests.
Check-worthy Factual Statement (CFS)(0.987)
But longer, hotter summers will 

In [29]:
get_claims_from_text(get_text_from_url("https://www.nationalgeographic.com/environment/article/amazon-rainforest-now-appears-to-be-contributing-to-climate-change"), threshold=0.5, debug=True);

Check-worthy Factual Statement (CFS)(0.972)
The Amazon rainforest is most likely now a net contributor to warming of the planet, according to a first-of-its-kind analysis from more than 30 scientists.
Check-worthy Factual Statement (CFS)(0.978)
For years, researchers have expressed concern that rising temperatures, drought, and deforestation are reducing the capacity of the world’s largest rainforest to absorb carbon dioxide from the atmosphere, and help offset emissions from fossil-fuel burning.
Check-worthy Factual Statement (CFS)(0.972)
Recent studies have even suggested that some portions of the tropical landscape already may release more carbon than they store.
Check-worthy Factual Statement (CFS)(0.480)
But the inhaling and exhaling of CO2 is just one way this damp jungle, the most species-rich on Earth, influences the global climate.
Check-worthy Factual Statement (CFS)(0.594)
Activities in the Amazon, both natural and human-caused, can shift the rainforest’s contribution in sig

## Is a sentence about climate?

### Works well on full texts

In [30]:
sentences = [get_text_from_url("https://news.sky.com/story/matt-hancock-responds-to-leak-of-lockdown-whatsapp-messages-12823380")]
filter_climate_related(sentences, debug=True);

Matt Hancock has denounced what he said was a "massive betrayal and breach of trust" following the leaking of lockdown WhatsApp messages.

The exchanges were published in the Daily Telegraph after he shared them with journalist Isabel Oakeshott, who worked with the former health secretary on his Pandemic Diaries book.

In a lengthy statement, Mr Hancock denied sending a "menacing message" to Ms Oakeshott after the publication of the first stories on Wednesday - a claim she made last night as she defended breaking a non-disclosure agreement (NDA) to leak the messages.

Image: Matt Hancock and Isabel Oakeshott. Pic: Parsons Media

The MP said: "I am hugely disappointed and sad at the massive betrayal and breach of trust by Isabel Oakeshott. I am also sorry for the impact on the very many people - political colleagues, civil servants and friends - who worked hard with me to get through the pandemic and save lives.

"There is absolutely no public interest case for this huge breach. All the

### Doesn't work so well on separate phrases

In [31]:
sentences = get_claims_from_text(get_text_from_url("https://news.sky.com/story/matt-hancock-responds-to-leak-of-lockdown-whatsapp-messages-12823380"))
filter_climate_related(sentences, debug=True);

Matt Hancock has denounced what he said was a "massive betrayal and breach of trust" following the leaking of lockdown WhatsApp messages.
climate related? Yes
prob: 0.996
The exchanges were published in the Daily Telegraph after he shared them with journalist Isabel Oakeshott, who worked with the former health secretary on his Pandemic Diaries book.
climate related? No
prob: 0.984
In a lengthy statement, Mr Hancock denied sending a "menacing message" to Ms Oakeshott after the publication of the first stories on Wednesday - a claim she made last night as she defended breaking a non-disclosure agreement (NDA) to leak the messages.
climate related? No
prob: 0.996
Image: Matt Hancock and Isabel Oakeshott.
climate related? No
prob: 0.991
Pic: Parsons Media

The MP said: "I am hugely disappointed and sad at the massive betrayal and breach of trust by Isabel Oakeshott.
climate related? No
prob: 0.978
All the materials for the book have already been made available to the inquiry, which is the 

## Semantic search (Looks pretty reasonable)

In [32]:
claim = "CO2 is not the cause of our current warming trend"
evidences = retrieve_matches_for_a_phrase(claim)
print('Claim\n', claim)
for i, evidence_doc in enumerate(evidences):
    print(f'Evidence {i}:\n',
          f'Similarity: {evidence_doc.score:.3f}\n'
          f'Quote: {evidence_doc.content}\n',
          f'Article Title: {evidence_doc.meta.get("title", "")}\n',
          f'DOI: {evidence_doc.meta.get("doi", "")}\n')

Claim
 CO2 is not the cause of our current warming trend
Evidence 0:
 Similarity: 0.842
Quote: Coal-fired power plants emit large amounts of CO 2 , which constitutes one the largest causes global warming.
 Article Title: COAL-FIRED POWER PLANTS ENERGY EFFICIENCY AND CLIMATE CHANGE-CURRENT STATE AND FUTURE
 DOI: 

Evidence 1:
 Similarity: 0.832
Quote: recent decades, United Nations’ (WMO, UNEP) change actions, IPCC reports, other scientific publications strongly emphasized that CO2 increase was “very likely” triggering global warming, which already led negative consequences for environment society.
 Article Title: Climate Change and Food Security Current and Future
 DOI: https://doi.org/10.1007/978-3-319-96256-6_9

Evidence 2:
 Similarity: 0.821
Quote: Our results support notion occur relatively soon after shifts then progressively approach zero.
 Article Title: Non-deforestation fire vs. fossil fuel combustion: the source of CO&amp;lt;sub&amp;gt;2&amp;lt;/sub&amp;gt; emissions affects 

In [33]:
claim = "The Amazon rainforest is most likely now a net contributor to warming of the planet, according to a first-of-its-kind analysis from more than 30 scientists."
evidences = retrieve_matches_for_a_phrase(claim)
print('Claim\n', claim)
for i, evidence_doc in enumerate(evidences):
    print(f'Evidence {i}:\n',
          f'Similarity: {evidence_doc.score:.3f}\n'
          f'Quote: {evidence_doc.content}\n',
          f'Article Title: {evidence_doc.meta.get("title", "")}\n',
          f'DOI: {evidence_doc.meta.get("doi", "")}\n')

Claim
 The Amazon rainforest is most likely now a net contributor to warming of the planet, according to a first-of-its-kind analysis from more than 30 scientists.
Evidence 0:
 Similarity: 0.840
Quote: Forest inventories from the intact rainforests of Amazon indicate increasing rates carbon gain over past three decades.
 Article Title: Evaluating the impacts of climate and elevated carbon dioxide on tropical rainforests of the western Amazon basin using ecosystem models and satellite data
 DOI: https://doi.org/10.1111/j.1365-2486.2009.01921.x

Evidence 1:
 Similarity: 0.832
Quote: Fires in tropical forests release globally significant amounts of carbon to the atmosphere and may increase importance as a result climate change.
 Article Title: Simulating fire regimes in the Amazon in response to climate change and deforestation
 DOI: https://doi.org/10.1890/10-0827.1

Evidence 2:
 Similarity: 0.831
Quote: In addition to process uncontrolled expansion agriculture frontier, Amazon, for its 

## Claim verification

In [34]:
input = [['The Amazon rainforest is most likely now a net contributor to warming of the planet, according to a first-of-its-kind analysis from more than 30 scientists.',
  ['Forest inventories from the intact rainforests of Amazon indicate increasing rates carbon gain over past three decades.',
   'Fires in tropical forests release globally significant amounts of carbon to the atmosphere and may increase importance as a result climate change.',
   'The Amazon forests are one of the largest ecosystem carbon pools on Earth.',
   'show that latter accounts for 35% reduced uptake (9 ± 10 vs. 26 7 g/m2/yr; mean 1 sd) 12% warming (0.09 0.03 0.7 0.07 K) during late 21st century.',
   '&amp;lt;p&amp;gt;The Amazon forests are one of the largest ecosystem carbon pools on Earth.',
   'Despite a fall in deforestation, frequency and severity of fires the Brazilian Amazon are rising, causing huge carbon emissions, biodiversity losses local economic costs.',
   'cause huge carbon emissions, biodiversity losses local economic costs.',
   'BAU PAs 19% over next four decades, while change alone account 12% increase.',
   'We first investigated NDVI found a positive trend during period, but was only months August December.',
   'Committed emissions under global warming uncurbed amount 21 +/- 4 Pg 2050.']],
 ['For years, researchers have expressed concern that rising temperatures, drought, and deforestation are reducing the capacity of the world’s largest rainforest to absorb carbon dioxide from the atmosphere, and help offset emissions from fossil-fuel burning.',
  ['Forest inventories from the intact rainforests of Amazon indicate increasing rates carbon gain over past three decades.',
   'Since then, high atmospheric has stabilized forests promoting plant growth, despite aridity.',
   '&amp;lt;p&amp;gt;The Amazon forests are one of the largest ecosystem carbon pools on Earth.',
   'Forests play increasing significant roles in regulating carbon balance mitigating change.',
   'The Amazon forests are one of the largest ecosystem carbon pools on Earth.',
   'Fires in tropical forests release globally significant amounts of carbon to the atmosphere and may increase importance as a result climate change.',
   "However, decrease 20%~40% most South Africa, forests' reduced 40%.",
   "The international legal framework for valuing the carbon stored in forests, known as 'Reducing Emissions from Deforestation and Forest Degradation' (REDD+), will have a major impact on indigenous peoples forest communities.",
   '[1]\xa0Numerous studies with general circulation models suggest that tropical deforestation can result in regional-scale climate change; namely, increased air temperature and wind speed reduced precipitation relative humidity.',
   'show that latter accounts for 35% reduced uptake (9 ± 10 vs. 26 7 g/m2/yr; mean 1 sd) 12% warming (0.09 0.03 0.7 0.07 K) during late 21st century.']],
 ['Recent studies have even suggested that some portions of the tropical landscape already may release more carbon than they store.',
  ['Globally, soils contain two or three times as much carbon living biomass.',
   'Our models project total ecosystem 13-24% potential global We present maps these altered distributions.',
   'A new report quantifies the carbon stored aboveground in tropical forests that are legally owned or traditionally held by Indigenous Peoples and local communities 37 countries across America, Africa, Asia.',
   'Soil 6-18% result combined changes.',
   'A modified version the Rothamsted Turnover model was used to evaluate response carbon, globally, changes atmospheric CO[sub 2] climate.',
   'Forest inventories from the intact rainforests of Amazon indicate increasing rates carbon gain over past three decades.',
   'current concentration, from set five GCMs yield 5-10% decreases despite general inputs.',
   '2.1 1980s 3.6 occurs both northern middle-high latitudes tropical regions.',
   'tropical zones had ability increase peak levels quickly, whereas temperate at slower rate peaked overall level.',
   'With no change climate, increased litter inputs due doubling resulted increases 11-22% depending choice function.']]]
labels, probs = predict_supports_or_refutes(input)

for label, prob in zip(labels, probs):
  print(label, prob)


SUPPORTS 0.44110891222953796
NOT_ENOUGH_INFO 0.8097778558731079
NOT_ENOUGH_INFO 0.7244937419891357
NOT_ENOUGH_INFO 0.9168328642845154
NOT_ENOUGH_INFO 0.632353663444519
NOT_ENOUGH_INFO 0.4640432298183441
NOT_ENOUGH_INFO 0.9115858674049377
NOT_ENOUGH_INFO 0.9664239287376404
NOT_ENOUGH_INFO 0.8711703419685364
NOT_ENOUGH_INFO 0.9341111779212952
SUPPORTS 0.5424025058746338
SUPPORTS 0.500881016254425
NOT_ENOUGH_INFO 0.8385871052742004
SUPPORTS 0.8651463985443115
SUPPORTS 0.7563245892524719
SUPPORTS 0.8456252217292786
NOT_ENOUGH_INFO 0.733542799949646
NOT_ENOUGH_INFO 0.685431957244873
SUPPORTS 0.6937985420227051
NOT_ENOUGH_INFO 0.8936540484428406
NOT_ENOUGH_INFO 0.916889488697052
NOT_ENOUGH_INFO 0.9488397836685181
NOT_ENOUGH_INFO 0.8366160988807678
NOT_ENOUGH_INFO 0.9677332639694214
NOT_ENOUGH_INFO 0.9693019986152649
NOT_ENOUGH_INFO 0.8870707750320435
NOT_ENOUGH_INFO 0.9166800379753113
NOT_ENOUGH_INFO 0.9618100523948669
NOT_ENOUGH_INFO 0.9248310327529907
NOT_ENOUGH_INFO 0.9594624638557434
